In [1]:
import requests

In [2]:
GENIUS_API_KEY = "<Enter_API_key_here>"
GENIUS_SEARCH_URL = "https://api.genius.com/search"

def search_genius_songs(query):
    headers = {"Authorization": f"Bearer {GENIUS_API_KEY}"}
    params = {"q": query}
    response = requests.get(GENIUS_SEARCH_URL, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()["response"]["hits"]
    return None

# query = "Shape of You Ed Sheeran"
# songs = search_genius_songs(query)
# for song in songs:
#     print(f"{song['result']['full_title']} - {song['result']['url']}")

In [3]:
!pip install beautifulsoup4


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [4]:
# from bs4 import BeautifulSoup

# def fetch_lyrics(genius_url):
#     response = requests.get(genius_url)
#     soup = BeautifulSoup(response.text, "html.parser")
#     lyrics_divs = soup.find_all("div", class_="Lyrics__Container")
#     lyrics = "\n".join([div.get_text("\n") for div in lyrics_divs])
#     return lyrics

# lyrics = fetch_lyrics(songs[0]['result']['url'])
# print(lyrics)


# import requests
# from bs4 import BeautifulSoup

# def get_genius_lyrics(url):
#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }

#     response = requests.get(url, headers=headers)
    
#     if response.status_code != 200:
#         print("Failed to retrieve the webpage")
#         return None

#     soup = BeautifulSoup(response.text, "html.parser")
    
#     # Find the div that contains the lyrics
#     lyrics_div = soup.find("div", class_="Lyrics__Container-sc-926d9e10-1 fEHzCI")
#     print("lyrics_div",lyrics_div)
#     if not lyrics_div:
#         print("Lyrics container not found")
#         return None

#     # Extract lyrics text
#     lyrics = "\n".join([line.get_text(separator="\n") for line in lyrics_div.find_all("p")])

#     return lyrics

# # Example usage
# url = "https://genius.com/Ed-sheeran-shape-of-you-lyrics"
# lyrics = get_genius_lyrics(url)
# if lyrics:
#     print(lyrics)

import requests
from bs4 import BeautifulSoup

def get_genius_lyrics(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print("Failed to retrieve the webpage")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all divs that contain lyrics
    lyrics_divs = soup.find_all("div", {"data-lyrics-container": "true"})

    if not lyrics_divs:
        print("Lyrics container not found")
        return None

    lyrics = []
    
    for div in lyrics_divs:
        # Extract text from normal text and <a> tags
        for elem in div.children:
            if elem.name == "a":  # Extract text from links
                lyrics.append(elem.get_text(separator="\n"))
            elif elem.name is None:  # Extract plain text
                lyrics.append(elem.strip())

    return "\n".join(lyrics)

# Example usage
# url = "https://genius.com/Ed-sheeran-shape-of-you-lyrics"
# lyrics = get_genius_lyrics(url)

# if lyrics:
#     print(lyrics)


In [5]:
!pip install yt-dlp

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [6]:
import yt_dlp
import os
import re

def search_youtube(query):
    ydl_opts = {
        "quiet": True,
        "format": "bestaudio/best",
        "default_search": "ytsearch",
        "noplaylist": True
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        search_result = ydl.extract_info(f"ytsearch:{query}", download=False)
        if search_result and "entries" in search_result and len(search_result["entries"]) > 0:
            # return search_result["entries"][0]["webpage_url"]  # Use webpage_url for actual video URL
            first_entry = search_result["entries"][0]
            return first_entry["webpage_url"], first_entry["title"]  # Return URL and Title
    return None, None

# youtube_url = search_youtube("Shape of You Ed Sheeran")
# print(youtube_url)

In [7]:
def sanitize_filename(filename):
    """Remove invalid characters from filename"""
    return re.sub(r'[\\/*?:"<>|]', "", filename)

In [8]:
# def download_song(youtube_url, output_path="downloads/song.mp3"):
#     ydl_opts = {
#         "format": "bestaudio/best",
#         "outtmpl": output_path,
#         "postprocessors": [{
#             "key": "FFmpegExtractAudio",
#             "preferredcodec": "mp3",
#             "preferredquality": "192",
#         }],
#     }
#     with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#         ydl.download([youtube_url])

# download_song(youtube_url)
# print("done")

def download_song(youtube_url, song_name):
    song_name = sanitize_filename(song_name)  # Clean up the song name for file safety
    output_path = f"downloads/{song_name}"  # Set the output filename

    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": output_path,
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192",
        }],
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])
    
    print(f"Download complete: {output_path}")

# Test

In [9]:
# query = "Love me like you do"
# songs = search_genius_songs(query)
# print("Link - ",songs[0]['result']['url'])
# url = songs[0]['result']['url']
# lyrics = get_genius_lyrics(url)
# print()
# if lyrics:
#     print(lyrics)

# youtube_url, song_name = search_youtube(query)

# if youtube_url and song_name:
#     if not os.path.exists("downloads"):
#         os.makedirs("downloads")  # Ensure downloads directory exists

#     download_song(youtube_url, song_name)
# else:
#     print("Song not found on YouTube.")

TypeError: 'NoneType' object is not subscriptable

In [ ]:
def get_genius_url(song_id: str) -> str:
    return f"https://genius.com/songs/{song_id}"

def download_and_store_lyrics(song_id: str) -> bool:
    # print(f"[Lyrics Scraper] Fetching lyrics for Genius song ID: {song_id}")
    genius_url = get_genius_url(song_id)

    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    try:
        response = requests.get(genius_url, headers=headers)
        if response.status_code != 200:
            print(f"[Lyrics Scraper] Failed to fetch Genius page for {song_id}")
            return False

        soup = BeautifulSoup(response.text, "html.parser")
        lyrics_divs = soup.find_all("div", {"data-lyrics-container": "true"})
        if not lyrics_divs:
            # print(f"[Lyrics Scraper] No lyrics container found for {song_id}")
            return False

        lyrics = []

        lyrics_divs = soup.find_all("div", {"data-lyrics-container": "true"})

        for div in lyrics_divs:
            # Flatten all inline elements inside the div into one single line
            # line = ''.join(div.strings).strip()
            line = ' '.join(s.strip() for s in div.strings if s.strip())

            # Remove empty lines and annotation headers like [Chorus]
            if not line or re.match(r"^\[.*\]$", line):
                continue

            # Remove lines with only punctuation or special characters
            if not re.search(r"[a-zA-Z0-9]", line):
                continue

            lyrics.append(line)

        # lyrics_text = "\n".join(filter(None, (line.strip() for line in lyrics)))
        lyrics_text = "\n".join(lyrics)

        # print(f"[Lyrics Scraper] Uploaded lyrics.txt")
        print(lyrics_text)
        with open("lyrics.txt", "w", encoding="utf-8") as f:
            f.write(lyrics_text)
        return True

    except Exception as e:
        print(f"[Lyrics Scraper] Unexpected error while fetching for {song_id}: {e}")
        return False

In [40]:
download_and_store_lyrics(5215007)

165 Contributors Translations Türkçe Español فارسی Deutsch Français Nederlands Polski Česky Italiano Levitating Lyrics [Verse 1] If you wanna run away with me, I know a galaxy And I can take you for a ride I had a premonition that we fell into a rhythm Where the music don't stop for life Glitter in the sky, glitter in my eyes Shining just the way I like If you're feeling like you need a little bit of company You met me at the perfect time [Pre-Chorus] You want me, I want you, baby My sugarboo, I'm levitating The Milky Way, we're renegading Yeah, yeah, yeah, yeah, yeah [Chorus] I got you, moonlight, you're my starlight I need you all night, come on, dance with me I'm levitating You, moonlight, you're my starlight ( You're the moonlight ) I need you all night, come on , dance with me I'm levitating [Verse 2] I believe that you're for me, I feel it in our energy I see us written in the stars We can go wherever, so let's do it now or never Baby, nothing's ever, ever too far Glitter in the 

True

In [61]:
import re
import requests
from bs4 import BeautifulSoup

def get_genius_url(song_id: str) -> str:
    return f"https://genius.com/songs/{song_id}"

def download_and_store_lyrics(song_id: str) -> bool:
    genius_url = get_genius_url(song_id)
    headers = {"User-Agent": "Mozilla/5.0"}
    
    try:
        response = requests.get(genius_url, headers=headers)
        if response.status_code != 200:
            print(f"[Lyrics Scraper] Failed to fetch Genius page for {song_id}")
            return False
        
        soup = BeautifulSoup(response.text, "html.parser")
        
        # First, try to find the newer format containers
        lyrics_containers = soup.find_all("div", class_=re.compile("Lyrics__Container"))
        
        # Fallback: older pages sometimes use a 'lyrics' container
        if not lyrics_containers:
            older_container = soup.find("div", class_="lyrics")
            if older_container:
                lyrics_containers = [older_container]
        
        if not lyrics_containers:
            print(f"[Lyrics Scraper] No lyrics container found for {song_id}")
            return False

        # Unwanted keywords to filter out entire blocks (if needed)
        unwanted_keywords = [
            'Contributors', 'Translations', 'Read More',
            'Türkçe', 'Español', 'فارسی', 'Deutsch',
            'Français', 'Nederlands', 'Polski', 'Česky', 'Italiano'
        ]
        
        lyrics_parts = []
        for container in lyrics_containers:
            # Replace <br> tags with a temporary marker.
            for br in container.find_all("br"):
                br.replace_with("<<<BR>>>")
            # Get the text by joining inline elements with a space
            text = container.get_text(separator=" ").strip()
            # Replace our marker with a newline so only intended breaks appear.
            text = text.replace("<<<BR>>>", "\n")
            # Skip container if any unwanted keyword is in its text.
            if any(kw in text for kw in unwanted_keywords):
                continue
            if text:
                lyrics_parts.append(text)

        # Join all sections with double newlines.
        lyrics_text = "\n\n".join(lyrics_parts)
        # Remove any content within square brackets (e.g. [Pre-Chorus], [Chorus]).
        lyrics_text = re.sub(r'\[[^\]]*\]', '', lyrics_text)
        # Clean up extra spaces and newlines that might result.
        lyrics_text = re.sub(r'\n+', '\n', lyrics_text).strip()
        lyrics_text = re.sub(r'\s+\n', '\n', lyrics_text)

        print(lyrics_text)
        with open("lyrics.txt", "w", encoding="utf-8") as f:
            f.write(lyrics_text)
        return True
    
    except Exception as e:
        print(f"[Lyrics Scraper] Unexpected error while fetching for {song_id}: {e}")
        return False


# Example usage:
# download_and_store_lyrics(5215007) # leviating
download_and_store_lyrics(77782) # yellow

( Aah ) And your skin, oh, yeah, your skin and bones
 ( Ooh ) Turn into something beautiful
 ( Aah ) And you know, for you, I'd bleed myself dry
 For you, I'd bleed myself dry
 It's true
 Look how they shine for you
 Look how they shine for you
 Look how they shine for
 Look how they shine for you
 Look how they shine for you
 Look how they shine
 Look at the stars
 Look how they shine for you
 And all the things that you do


True

Opening URL: https://genius.com/Coldplay-yellow-lyrics
Error during Selenium processing: Message: 

No page source retrieved.



Look at the stars
look how they shine for you
and everything you do
yeah they were all yellow
I came along 
I wrote a song for you

and all the things you do

and it was called yellow

So then I took my turn 

oh what a thing to have done

and it was all yellow



your skin

oh yeah your skin and bones

turn into something beautiful

and you know

you know I love you so

you know I love you so



I swam across

I jumped across for you

oh what a thing to do

'cos you were all yellow

I drew a line

I drew a line for you

oh what a thing to do

and it was all yellow



and your skin

oh yeah your skin and bones

turn into something beautiful

and you know

for you I bleed myself dry

for you I bleed myself dry



it's true

look how they shine for you

look how they shine for you

look how they shine for

look how they shine for you

look how they shine for you

look how they shine



look at the stars

look how they shine for you

and all the things that you do


In [86]:
import re
import requests
from bs4 import BeautifulSoup

def build_azlyrics_url(artist, title):
    """Build a URL to AZLyrics using only the main artist."""
    # Extract only the first artist (before ft., feat., &, x, etc.)
    artist = re.split(r'\s*(?:ft\.?|feat\.?|featuring|&|,|/|\+|x)\s*', artist, flags=re.IGNORECASE)[0]

    def clean(string):
        return re.sub(r'[^a-z0-9]', '', string.lower())
    
    artist = clean(artist)
    title = clean(title)
    return f"https://www.azlyrics.com/lyrics/{artist}/{title}.html"

def scrape_azlyrics(url):
    """Scrape lyrics from AZLyrics page."""
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)

    if res.status_code != 200:
        raise Exception(f"Failed to fetch page: {res.status_code}")
    
    soup = BeautifulSoup(res.text, "html.parser")
    divs = soup.find_all("div", attrs={"class": None, "id": None})
    for div in divs:
        if div.text.strip():
            return div.text.strip()
    return "Lyrics not found."

# # Try it
# title, artist = "See You Again", "Wiz Khalifa ft. Charlie Puth"
# title, artist = "SNTMNG", "Sfera Ebbasta & Shiva"
# title, artist = "Lean On", "Major Lazer & DJ Shake ft. MØ"
# title, artist = "Sugar", "Maroon 5"
# title, artist = "What Do You Mean?", "Justin Bieber"
title, artist = "Despacito (Remix)", "Luis Fonsi"

print(f"Title: {title}, Artist: {artist}")

az_url = build_azlyrics_url(artist, title)
print(f"Fetching from: {az_url}")

lyrics = scrape_azlyrics(az_url)
print("\nLyrics:\n", lyrics)


Title: Despacito (Remix), Artist: Luis Fonsi
Fetching from: https://www.azlyrics.com/lyrics/luisfonsi/despacitoremix.html

Lyrics:
 [Justin Bieber:]
Coming over in my direction
So thankful for that, it's such a blessin', yeah
Turn every situation into Heaven, yeah
Oh, you are my sunrise on the darkest day
Got me feelin' some kind of way
Make me wanna savour every moment slowly, slowly
You fit me tailor-made, love, how you put it on
Got the only key, know how to turn it on
The way you nibble on my ear,
The only words I wanna hear
Baby, take it slow so we can last long

[Luis Fonsi:]
Oh, tÃº, tÃº eres el imÃ¡n y yo soy el metal
Me voy acercando y voy armando el plan
SÃ³lo con pensarlo se acelera el pulso

[Daddy Yankee:]
Oh, yeah ya, ya me estÃ¡ gustando mÃ¡s de lo normal
Todos mis sentidos van pidiendo mÃ¡s
Esto hay que tomarlo sin ningÃºn apuro

[Justin Bieber:]
Despacito
Quiero respirar tu cuello despacito
Deja que te diga cosas al oÃ­do
Para que te acuerdes si no estÃ¡s conmigo
Despa